# 순서
* 모델 구축
* 입력값 토큰화, 임베딩
* 훈련을 정의하고 학습
* 평가
* 예측하기 위한 데이터를 토큰화, 임베딩후 predict
* 라벨 출력

In [1]:
import tensorflow as tf
from tensorflow.keras import Model, layers

# 인코더 입력 정의: 영문 단어가 입력, 각 단어는 4자 길이, 모든 문자의 수는 172개
enc_input = layers.Input(shape=(4, 172))

# 인코더 LSTM 정의: 모든 타임스텝의 출력을 반환하도록 설정
enc_output, state_h, state_c = layers.LSTM(128, return_sequences=True, return_state=True)(enc_input)

# 디코더 입력 정의
dec_input = layers.Input(shape=(3, 172))  # 한글 단어는 2자 길이 + <Start> 토큰, 모든 문자의 수는 171개

# 디코더 LSTM 정의: 모든 시퀀스의 출력을 반환하도록 설정
dec_lstm_output, _, _ = layers.LSTM(128, return_sequences=True, return_state=True)(dec_input, initial_state=[state_h, state_c])

# 어텐션 메커니즘 정의
context_vector = layers.Attention()([dec_lstm_output, enc_output])

# 컨텍스트 벡터와 디코더 LSTM 출력을 결합
context_and_lstm_output = layers.Concatenate()([context_vector, dec_lstm_output])

# 디코더 출력층 정의: 출력 크기는 모든 문자의 수인 171, softmax 활성화 함수를 사용
output = layers.Dense(172, activation='softmax')(context_and_lstm_output)

# 모델 정의: 인코더 입력(enc_input)과 디코더 입력(dec_input)을 모델의 입력으로, 디코더 출력을 모델의 출력으로 설정
model = Model(inputs=[enc_input, dec_input], outputs=[output])

# 모델 요약 출력
model.summary()


2025-12-05 11:25:34.206024: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-05 11:25:34.206372: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-05 11:25:34.249080: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-05 11:26:47.210369: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 4, 172)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 3, 172)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 4, 128),  │    154,112 │ input_layer[0][0] │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 3, 128),  │    154,112 │ input_layer_1[0]… │
│                     │ (None, 128),      │            │ lstm[0][1],       │
│                     │ (None, 128)]      │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 3, 128)    │          0 │ lstm_1[0][0],     │
│ (Attention)         │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 3, 256)    │          0 │ attention[0][0],  │
│ (Concatenate)       │                   │            │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 3, 172)    │     44,204 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 352,428 (1.34 MB)

 Trainable params: 352,428 (1.34 MB)

 Non-trainable params: 0 (0.00 B)

In [2]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam') # metrics가 없으면 훈련시 loss만 출력, predict()를 사용 못함

In [3]:
import pandas as pd
import numpy as np
# <START>, <END>, <PAD>
arr1 = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
arr2 = pd.read_csv('korean.csv', header=None)
# print(arr2[0].values.tolist())
num_to_char = arr1 + arr2[0].values.tolist()
print(len(num_to_char))

171


In [4]:
char_to_num = {char:i for i, char in enumerate(num_to_char)}
# print(char_to_num)

In [5]:
raw = pd.read_csv('translate.csv', header=None)
eng_kor = raw.values.tolist()
print(len(eng_kor))

110


In [6]:
temp_eng = 'love'
temp_eng_n = [char_to_num[c] for c in temp_eng]
print(temp_eng_n)
temp_kor = '사랑'
np.eye(171)[temp_eng_n].shape

[14, 17, 24, 7]


(4, 171)

In [7]:
def encode(eng_kor):
    enc_in = []
    dec_in = []
    rnn_out = [] # decoder output
    for seq in eng_kor:
        eng = [char_to_num[c] for c in seq[0]]
        enc_in.append(np.eye(172)[eng])

        kor = [char_to_num[c] for c in ('S'+seq[1])]
        dec_in.append(np.eye(172)[kor])

        target = [char_to_num[c] for c in (seq[1] + 'E')]
        rnn_out.append(target)

    enc_in = np.array(enc_in)
    dec_in = np.array(dec_in)
    rnn_out = np.array(rnn_out)
    rnn_out = np.expand_dims(rnn_out, axis=2)
    return enc_in, dec_in, rnn_out

In [8]:
sample = [['word', '단어']]
encode(sample)[2]

array([[[ 61],
        [114],
        [  1]]])

In [9]:
X_enc, X_dec, y_rnn = encode(eng_kor)

In [10]:
%%time
model.fit([X_enc, X_dec], y_rnn, epochs=500)

Epoch 1/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 5.1337
Epoch 2/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 5.0905
Epoch 3/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 5.0369
Epoch 4/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 4.9539
Epoch 5/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 4.8061
Epoch 6/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 4.5051
Epoch 7/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 3.9126
Epoch 8/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.5253
Epoch 9/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.6037
Epoch 10/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.4098
Epoch 11/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 3.3745
Epoch 12/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 3.3701
Epoch 13/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.3080
Epoch 14/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 3.2746
Epoch 15/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.2610
Epoch 16/500
4/4 ━━

In [11]:
enc_in, dec_in, _ = encode([['help', 'PP']])
# print(enc_in)
pred = model.predict([enc_in, dec_in])
# print(pred.shape)
word = np.argmax(pred[0], axis=-1)
# print(word)
num_to_char[word[0]], num_to_char[word[1]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step


('도', '움')

In [12]:
from numpy.random import randint
pick = randint(0, len(eng_kor), 5)
choose = [ [eng_kor[i][0], 'PP'] for i in pick]
print(choose)

[['wide', 'PP'], ['trip', 'PP'], ['farm', 'PP'], ['rose', 'PP'], ['weak', 'PP']]


In [13]:
enc_in, dec_in, _ = encode(choose)
pred = model.predict([enc_in, dec_in])
print(pred.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
(5, 3, 172)


In [14]:
for i in range(len(choose)):
    eng = choose[i][0]
    word = np.argmax(pred[i], axis=-1)
    kor = ''
    for j in range(2):
        kor = kor + num_to_char[word[j]]
    print(eng, kor)

wide 넓은
trip 여행
farm 농장
rose 장미
weak 약한
